In [1]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import gpxpy
import haversine
import json

# Read the config JSON file

In [2]:
with open("config.json", "r") as config_file:
    config = json.load(config_file)

# Read data from the CSV file

In [3]:
df = pd.read_csv("race_results.csv", 
                 parse_dates=["date"], 
                 date_format="%Y-%m-%d")

In [4]:
df

,name,distance,date,city,country,duration,pace,gpxfilename
0,Cursa Moritz Sant Antoni,5,2025-01-26,Barcelona,Spain,0:29:19,00:05:52,Cursa_Moritz_Sant_Antoni_5km_2025.gpx
1,XIII Cursa Benefica Malalties Minoritaries,5,2025-02-23,Badalona,Spain,00:27:10,0:05:26,Cursa_Benèfica_Malalties_Minoritàries_Badalona...
2,Cursa de la Dona Pienda,5,2025-03-23,Pineda de Mar,Spain,0:25:36,00:05:08,Cursa_de_la_Dona_Pineda_de_Mar_5_Km.gpx
3,15a Cursa Nocturna de L'Hospitalet,5,2025-04-05,L'Hospitalet de Llobregat,Spain,0:25:40,00:05:08,Cursa_Nocturna_de_L_Hospitalet_5_Km.gpx
4,Cursa Montilivi Girona,5,2025-04-27,Girona,Spain,0:28:49,0:05:46,Cursa_Montilivi_Girona_5_Km.gpx
5,La Cursa de RAC1 Port de Barcelona,6,2025-05-18,Barcelona,Spain,0:30:04,00:05:01,Cursa_de_RAC1_Port_de_Barcelona_6Km.gpx
6,14a Cursa Popular Illa Carlemany,5,2025-06-01,Andorra la Vella,Andorra,0:25:12,00:05:03,Cursa_Popular_Illa_Carlemany_Andorra_5_Km.gpx
7,45a Cursa Popular Ciutat de Badalona - La Curs...,5,2025-06-28,Badalona,Spain,0:23:40,00:04:44,La_Cursa_Del_Dimoni_Badalona_5Km.gpx


In [5]:
df.dtypes

name                   object
distance                int64
date           datetime64[ns]
city                   object
country                object
duration               object
pace                   object
gpxfilename            object
dtype: object

# Add column with date as string

In [6]:
df["date_str"] = df.apply(lambda row: row["date"].strftime("%b %d, %Y"), axis=1)

# Add columns with total duration converted into timedelta and seconds

In [7]:
def from_str_to_timedelta(row):
    duration = datetime.strptime(row["duration_total_timedelta"], "%H:%M:%S")
    duration = timedelta(hours=duration.hour,
                         minutes=duration.minute,
                         seconds=duration.second)
    return duration

In [8]:
df = df.rename(columns={"duration": "duration_total_timedelta"})
df["duration_total_timedelta"] = df.apply(from_str_to_timedelta, axis=1)

df["duration_total_sec"] = df.apply(lambda row: row["duration_total_timedelta"].seconds, axis=1)

In [9]:
df

,name,distance,date,city,country,duration_total_timedelta,pace,gpxfilename,date_str,duration_total_sec
0,Cursa Moritz Sant Antoni,5,2025-01-26,Barcelona,Spain,0 days 00:29:19,00:05:52,Cursa_Moritz_Sant_Antoni_5km_2025.gpx,"Jan 26, 2025",1759
1,XIII Cursa Benefica Malalties Minoritaries,5,2025-02-23,Badalona,Spain,0 days 00:27:10,0:05:26,Cursa_Benèfica_Malalties_Minoritàries_Badalona...,"Feb 23, 2025",1630
2,Cursa de la Dona Pienda,5,2025-03-23,Pineda de Mar,Spain,0 days 00:25:36,00:05:08,Cursa_de_la_Dona_Pineda_de_Mar_5_Km.gpx,"Mar 23, 2025",1536
3,15a Cursa Nocturna de L'Hospitalet,5,2025-04-05,L'Hospitalet de Llobregat,Spain,0 days 00:25:40,00:05:08,Cursa_Nocturna_de_L_Hospitalet_5_Km.gpx,"Apr 05, 2025",1540
4,Cursa Montilivi Girona,5,2025-04-27,Girona,Spain,0 days 00:28:49,0:05:46,Cursa_Montilivi_Girona_5_Km.gpx,"Apr 27, 2025",1729
5,La Cursa de RAC1 Port de Barcelona,6,2025-05-18,Barcelona,Spain,0 days 00:30:04,00:05:01,Cursa_de_RAC1_Port_de_Barcelona_6Km.gpx,"May 18, 2025",1804
6,14a Cursa Popular Illa Carlemany,5,2025-06-01,Andorra la Vella,Andorra,0 days 00:25:12,00:05:03,Cursa_Popular_Illa_Carlemany_Andorra_5_Km.gpx,"Jun 01, 2025",1512
7,45a Cursa Popular Ciutat de Badalona - La Curs...,5,2025-06-28,Badalona,Spain,0 days 00:23:40,00:04:44,La_Cursa_Del_Dimoni_Badalona_5Km.gpx,"Jun 28, 2025",1420


# Add columns with duration per km converted into timedelta, timedelta as string and seconds

In [10]:
df["duration_km_timedelta"] = df["duration_total_timedelta"] / df["distance"]

df["duration_km_timedelta_str"] = df.apply(lambda row: datetime.strftime(datetime(2025, 1, 1) +
                                                                         row["duration_km_timedelta"], "%H:%M:%S"),
                                           axis=1)

df["duration_km_sec"] = df["duration_total_sec"] / df["distance"]

In [11]:
df

,name,distance,date,city,country,duration_total_timedelta,pace,gpxfilename,date_str,duration_total_sec,duration_km_timedelta,duration_km_timedelta_str,duration_km_sec
0,Cursa Moritz Sant Antoni,5,2025-01-26,Barcelona,Spain,0 days 00:29:19,00:05:52,Cursa_Moritz_Sant_Antoni_5km_2025.gpx,"Jan 26, 2025",1759,0 days 00:05:51.800000,00:05:51,351.800000
1,XIII Cursa Benefica Malalties Minoritaries,5,2025-02-23,Badalona,Spain,0 days 00:27:10,0:05:26,Cursa_Benèfica_Malalties_Minoritàries_Badalona...,"Feb 23, 2025",1630,0 days 00:05:26,00:05:26,326.000000
2,Cursa de la Dona Pienda,5,2025-03-23,Pineda de Mar,Spain,0 days 00:25:36,00:05:08,Cursa_de_la_Dona_Pineda_de_Mar_5_Km.gpx,"Mar 23, 2025",1536,0 days 00:05:07.200000,00:05:07,307.200000
3,15a Cursa Nocturna de L'Hospitalet,5,2025-04-05,L'Hospitalet de Llobregat,Spain,0 days 00:25:40,00:05:08,Cursa_Nocturna_de_L_Hospitalet_5_Km.gpx,"Apr 05, 2025",1540,0 days 00:05:08,00:05:08,308.000000
4,Cursa Montilivi Girona,5,2025-04-27,Girona,Spain,0 days 00:28:49,0:05:46,Cursa_Montilivi_Girona_5_Km.gpx,"Apr 27, 2025",1729,0 days 00:05:45.800000,00:05:45,345.800000
5,La Cursa de RAC1 Port de Barcelona,6,2025-05-18,Barcelona,Spain,0 days 00:30:04,00:05:01,Cursa_de_RAC1_Port_de_Barcelona_6Km.gpx,"May 18, 2025",1804,0 days 00:05:00.666666666,00:05:00,300.666667
6,14a Cursa Popular Illa Carlemany,5,2025-06-01,Andorra la Vella,Andorra,0 days 00:25:12,00:05:03,Cursa_Popular_Illa_Carlemany_Andorra_5_Km.gpx,"Jun 01, 2025",1512,0 days 00:05:02.400000,00:05:02,302.400000
7,45a Cursa Popular Ciutat de Badalona - La Curs...,5,2025-06-28,Badalona,Spain,0 days 00:23:40,00:04:44,La_Cursa_Del_Dimoni_Badalona_5Km.gpx,"Jun 28, 2025",1420,0 days 00:04:44,00:04:44,284.000000


# Add columns with latitude, longitude, elevation and timestamp of route points from the .GPX file

In [12]:
def parse_gpx_file(filepath):
    lat = []
    lon = []
    elev = []
    timestamp = []

    with open(filepath, 'r') as gpx_file:
        gpx = gpxpy.parse(gpx_file)
        for track in gpx.tracks:
            for segment in track.segments:
                for point in segment.points:
                    lat.append(point.latitude)
                    lon.append(point.longitude)
                    elev.append(point.elevation)
                    timestamp.append(point.time)

    return lat, lon, elev, timestamp

In [13]:
df["route_points_lat"] = None
df["route_points_lon"] = None
df["route_points_elev"] = None
df["route_points_timestamp"] = None

for idx_row, row in df.iterrows():
    lat, lon, elev, timestamp = parse_gpx_file(config["gpx_race_route_filepath"] + row["gpxfilename"])
    df.at[idx_row, "route_points_lat"] = np.array(lat)
    df.at[idx_row, "route_points_lon"] = np.array(lon)
    df.at[idx_row, "route_points_elev"] = np.array(elev)
    df.at[idx_row, "route_points_timestamp"] = np.array(timestamp)

In [14]:
df

,name,distance,date,city,country,duration_total_timedelta,pace,gpxfilename,date_str,duration_total_sec,duration_km_timedelta,duration_km_timedelta_str,duration_km_sec,route_points_lat,route_points_lon,route_points_elev,route_points_timestamp
0,Cursa Moritz Sant Antoni,5,2025-01-26,Barcelona,Spain,0 days 00:29:19,00:05:52,Cursa_Moritz_Sant_Antoni_5km_2025.gpx,"Jan 26, 2025",1759,0 days 00:05:51.800000,00:05:51,351.800000,"[41.377388, 41.377388, 41.37739, 41.377391, 41...","[2.156912, 2.156912, 2.156909, 2.156909, 2.156...","[30.2, 30.2, 30.2, 30.2, 30.2, 30.2, 30.2, 30....","[2025-01-26 09:43:12+00:00, 2025-01-26 09:43:1..."
1,XIII Cursa Benefica Malalties Minoritaries,5,2025-02-23,Badalona,Spain,0 days 00:27:10,0:05:26,Cursa_Benèfica_Malalties_Minoritàries_Badalona...,"Feb 23, 2025",1630,0 days 00:05:26,00:05:26,326.000000,"[41.448694, 41.448694, 41.44871, 41.448723, 41...","[2.25207, 2.25207, 2.252083, 2.252094, 2.25210...","[9.1, 9.1, 9.1, 9.1, 9.1, 9.1, 9.1, 9.1, 9.1, ...","[2025-02-23 08:42:55+00:00, 2025-02-23 08:42:5..."
2,Cursa de la Dona Pienda,5,2025-03-23,Pineda de Mar,Spain,0 days 00:25:36,00:05:08,Cursa_de_la_Dona_Pineda_de_Mar_5_Km.gpx,"Mar 23, 2025",1536,0 days 00:05:07.200000,00:05:07,307.200000,"[41.62281, 41.622792, 41.622783, 41.622775, 41...","[2.676377, 2.676385, 2.676388, 2.676391, 2.676...","[14.0, 14.0, 14.0, 13.9, 13.9, 13.9, 13.8, 13....","[2025-03-23 09:01:55+00:00, 2025-03-23 09:01:5..."
3,15a Cursa Nocturna de L'Hospitalet,5,2025-04-05,L'Hospitalet de Llobregat,Spain,0 days 00:25:40,00:05:08,Cursa_Nocturna_de_L_Hospitalet_5_Km.gpx,"Apr 05, 2025",1540,0 days 00:05:08,00:05:08,308.000000,"[41.356897, 41.356885, 41.356867, 41.356855, 4...","[2.125731, 2.125723, 2.125715, 2.125708, 2.125...","[11.9, 11.9, 12.0, 12.0, 12.0, 12.0, 12.0, 12....","[2025-04-05 19:02:53+00:00, 2025-04-05 19:02:5..."
4,Cursa Montilivi Girona,5,2025-04-27,Girona,Spain,0 days 00:28:49,0:05:46,Cursa_Montilivi_Girona_5_Km.gpx,"Apr 27, 2025",1729,0 days 00:05:45.800000,00:05:45,345.800000,"[41.960944, 41.960946, 41.960957, 41.960969, 4...","[2.827218, 2.827183, 2.827172, 2.827162, 2.827...","[105.0, 105.0, 104.9, 104.9, 104.9, 104.9, 104...","[2025-04-27 07:47:48+00:00, 2025-04-27 07:47:4..."
5,La Cursa de RAC1 Port de Barcelona,6,2025-05-18,Barcelona,Spain,0 days 00:30:04,00:05:01,Cursa_de_RAC1_Port_de_Barcelona_6Km.gpx,"May 18, 2025",1804,0 days 00:05:00.666666666,00:05:00,300.666667,"[41.375038, 41.375052, 41.375067, 41.375081, 4...","[2.178087, 2.178092, 2.178096, 2.178101, 2.178...","[8.0, 8.0, 8.0, 8.1, 8.1, 8.1, 8.1, 8.2, 8.3, ...","[2025-05-18 07:05:06+00:00, 2025-05-18 07:05:0..."
6,14a Cursa Popular Illa Carlemany,5,2025-06-01,Andorra la Vella,Andorra,0 days 00:25:12,00:05:03,Cursa_Popular_Illa_Carlemany_Andorra_5_Km.gpx,"Jun 01, 2025",1512,0 days 00:05:02.400000,00:05:02,302.400000,"[42.509189, 42.509189, 42.509118, 42.509119, 4...","[1.534211, 1.534211, 1.534209, 1.534194, 1.534...","[1028.0, 1028.0, 1028.7, 1028.7, 1028.8, 1028....","[2025-06-01 08:31:13+00:00, 2025-06-01 08:31:1..."
7,45a Cursa Popular Ciutat de Badalona - La Curs...,5,2025-06-28,Badalona,Spain,0 days 00:23:40,00:04:44,La_Cursa_Del_Dimoni_Badalona_5Km.gpx,"Jun 28, 2025",1420,0 days 00:04:44,00:04:44,284.000000,"[41.449118, 41.44913, 41.449141, 41.449153, 41...","[2.252589, 2.2526, 2.252611, 2.252623, 2.25263...","[9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 8.9, 8.9, 8.9, ...","[2025-06-28 18:01:43+00:00, 2025-06-28 18:01:4..."


# Add columns with distance and duration for each route step
# Add columns with elapsed time in seconds, timedelta and timedelta as string
# Add columns with covered distance in km

In [16]:
df["route_points_dist_step_km"] = None
df["route_points_dist_accum_km"] = None
df["route_points_duration_step_sec"] = None
df["route_points_duration_accum_sec"] = None
df["route_points_duration_accum_timedelta"] = None
df["route_points_duration_accum_timedelta_str"] = None
df["route_points_dist_accum_percentage"] = None

for idx_row, row in df.iterrows():
    dist_step_km = [0]
    duration_step_sec = [0]
    for idx_point in range(1, len(row["route_points_lat"])):
        dist_step_current_km = haversine.haversine(
            (row["route_points_lat"][idx_point - 1], row["route_points_lon"][idx_point - 1]),
            (row["route_points_lat"][idx_point], row["route_points_lon"][idx_point]))
        duration_step_current_sec = (row["route_points_timestamp"][idx_point] -
                                     row["route_points_timestamp"][idx_point - 1]).seconds

        dist_step_km.append(dist_step_current_km)
        duration_step_sec.append(duration_step_current_sec)

    dist_accum_km = np.cumsum(dist_step_km)
    calculate_dist_accum_percentage = np.vectorize(lambda elem: elem * 100 / dist_accum_km[-1])
    dist_accum_percentage = calculate_dist_accum_percentage(dist_accum_km)
    
    df.at[idx_row, "route_points_dist_step_km"] = np.array(dist_step_km)
    df.at[idx_row, "route_points_dist_accum_km"] = dist_accum_km
    df.at[idx_row, "route_points_duration_step_sec"] = np.array(duration_step_sec)
    df.at[idx_row, "route_points_duration_accum_sec"] = np.cumsum(duration_step_sec)
    df.at[idx_row, "route_points_duration_accum_timedelta"] = np.array(
        [timedelta(seconds=elem.item()) for elem in df.at[idx_row, "route_points_duration_accum_sec"]])
    df.at[idx_row, "route_points_duration_accum_timedelta_str"] = np.array(
        [datetime.strftime(datetime(2025, 1, 1) + elem, "%H:%M:%S") for elem in df.at[idx_row, "route_points_duration_accum_timedelta"]])
    df.at[idx_row, "route_points_dist_accum_percentage"] = dist_accum_percentage

In [17]:
df

,name,distance,date,city,country,duration_total_timedelta,pace,gpxfilename,date_str,duration_total_sec,...,route_points_lon,route_points_elev,route_points_timestamp,route_points_dist_step_km,route_points_dist_accum_km,route_points_duration_step_sec,route_points_duration_accum_sec,route_points_duration_accum_timedelta,route_points_duration_accum_timedelta_str,route_points_dist_accum_percentage
0,Cursa Moritz Sant Antoni,5,2025-01-26,Barcelona,Spain,0 days 00:29:19,00:05:52,Cursa_Moritz_Sant_Antoni_5km_2025.gpx,"Jan 26, 2025",1759,...,"[2.156912, 2.156912, 2.156909, 2.156909, 2.156...","[30.2, 30.2, 30.2, 30.2, 30.2, 30.2, 30.2, 30....","[2025-01-26 09:43:12+00:00, 2025-01-26 09:43:1...","[0.0, 0.0, 0.0003348342746602575, 0.0001111950...","[0.0, 0.0, 0.0003348342746602575, 0.0004460293...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0:00:00, 0:00:01, 0:00:02, 0:00:03, 0:00:04, ...","[00:00:00, 00:00:01, 00:00:02, 00:00:03, 00:00...","[0.0, 0.0, 0.006435506593734519, 0.00857267333..."
1,XIII Cursa Benefica Malalties Minoritaries,5,2025-02-23,Badalona,Spain,0 days 00:27:10,0:05:26,Cursa_Benèfica_Malalties_Minoritàries_Badalona...,"Feb 23, 2025",1630,...,"[2.25207, 2.25207, 2.252083, 2.252094, 2.25210...","[9.1, 9.1, 9.1, 9.1, 9.1, 9.1, 9.1, 9.1, 9.1, ...","[2025-02-23 08:42:55+00:00, 2025-02-23 08:42:5...","[0.0, 0.0, 0.002083085208115605, 0.00171175636...","[0.0, 0.0, 0.002083085208115605, 0.00379484157...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0:00:00, 0:00:01, 0:00:02, 0:00:03, 0:00:04, ...","[00:00:00, 00:00:01, 00:00:02, 00:00:03, 00:00...","[0.0, 0.0, 0.041504286537031976, 0.07561005728..."
2,Cursa de la Dona Pienda,5,2025-03-23,Pineda de Mar,Spain,0 days 00:25:36,00:05:08,Cursa_de_la_Dona_Pineda_de_Mar_5_Km.gpx,"Mar 23, 2025",1536,...,"[2.676377, 2.676385, 2.676388, 2.676391, 2.676...","[14.0, 14.0, 14.0, 13.9, 13.9, 13.9, 13.8, 13....","[2025-03-23 09:01:55+00:00, 2025-03-23 09:01:5...","[0.0, 0.002109085590838851, 0.0010313561785847...","[0.0, 0.002109085590838851, 0.0031404417694235...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0:00:00, 0:00:01, 0:00:02, 0:00:03, 0:00:04, ...","[00:00:00, 00:00:01, 00:00:02, 00:00:03, 00:00...","[0.0, 0.04204420472639872, 0.0626040864621635,..."
3,15a Cursa Nocturna de L'Hospitalet,5,2025-04-05,L'Hospitalet de Llobregat,Spain,0 days 00:25:40,00:05:08,Cursa_Nocturna_de_L_Hospitalet_5_Km.gpx,"Apr 05, 2025",1540,...,"[2.125731, 2.125723, 2.125715, 2.125708, 2.125...","[11.9, 11.9, 12.0, 12.0, 12.0, 12.0, 12.0, 12....","[2025-04-05 19:02:53+00:00, 2025-04-05 19:02:5...","[0.0, 0.0014920806727579063, 0.002109949572214...","[0.0, 0.0014920806727579063, 0.003602030244972...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0:00:00, 0:00:01, 0:00:02, 0:00:03, 0:00:04, ...","[00:00:00, 00:00:01, 00:00:02, 00:00:03, 00:00...","[0.0, 0.029095442491893327, 0.0702392744307443..."
4,Cursa Montilivi Girona,5,2025-04-27,Girona,Spain,0 days 00:28:49,0:05:46,Cursa_Montilivi_Girona_5_Km.gpx,"Apr 27, 2025",1729,...,"[2.827218, 2.827183, 2.827172, 2.827162, 2.827...","[105.0, 105.0, 104.9, 104.9, 104.9, 104.9, 104...","[2025-04-27 07:47:48+00:00, 2025-04-27 07:47:4...","[0.0, 0.002902498443740042, 0.0015242488363474...","[0.0, 0.002902498443740042, 0.0044267472800874...","[0, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[0:00:00, 0:00:01, 0:00:02, 0:00:04, 0:00:05, ...","[00:00:00, 00:00:01, 00:00:02, 00:00:04, 00:00...","[0.0, 0.04934799508987207, 0.07526312495119715..."
5,La Cursa de RAC1 Port de Barcelona,6,2025-05-18,Barcelona,Spain,0 days 00:30:04,00:05:01,Cursa_de_RAC1_Port_de_Barcelona_6Km.gpx,"May 18, 2025",1804,...,"[2.178087, 2.178092, 2.178096, 2.178101, 2.178...","[8.0, 8.0, 8.0, 8.1, 8.1, 8.1, 8

# Calculate and plot the pace

In [32]:
import math

df["pace_sec"] = None
df["pace_timedelta"] = None
df["pace_timedelta_str"] = None
df["pace_dist_km"] = None
df["pace_average_calc_sec"] = None
df["pace_average_calc_timedelta"] = None
df["pace_average_calc_timedelta_str"] = None

for idx_row, row in df.iterrows():
    dist_accum_km = row["route_points_dist_accum_km"]
    duration_accum_sec = row["route_points_duration_accum_sec"]
    
    pace_sec = []
    pace_timedelta = []
    pace_timedelta_str = []
    pace_dist_km = []
    last_km_mark = (None, None, None) # (km_mark, distance, time)
    
    for idx_step, step in enumerate(dist_accum_km):
        km_current = math.modf(step)[1]
        if km_current != last_km_mark[0]:
            if last_km_mark[0] is not None:
                duration_diff = duration_accum_sec[idx_step] - last_km_mark[2]
                dist_diff = step - last_km_mark[1]
                pace_sec.append(int(duration_diff / dist_diff))
                pace_timedelta.append(timedelta(seconds=pace_sec[-1]))
                pace_timedelta_str.append(datetime.strftime(datetime(2025, 1, 1) + pace_timedelta[-1], "%H:%M:%S"))
                pace_dist_km.append(dist_diff)
            last_km_mark = (km_current, step, duration_accum_sec[idx_step])
            
    duration_diff = duration_accum_sec[-1] - last_km_mark[2]
    dist_diff = dist_accum_km[-1] - last_km_mark[1]
    pace_sec.append(int(duration_diff / dist_diff))
    pace_dist.append(dist_diff)
    
    df.at[idx_row, "pace_sec"] = np.array(pace_sec)
    df.at[idx_row, "pace_timedelta"] = np.array(pace_timedelta)
    df.at[idx_row, "pace_timedelta_str"] = np.array(pace_timedelta_str)
    df.at[idx_row, "pace_dist_km"] = np.array(pace_dist_km)

    pace_average = duration_accum_sec[-1] / dist_accum_km[-1]
    df.at[idx_row, "pace_average_calc_sec"] = int(pace_average)
    df.at[idx_row, "pace_average_calc_timedelta"] = timedelta(seconds=int(pace_average))
    df.at[idx_row, "pace_average_calc_timedelta_str"] = datetime.strftime(datetime(2025, 1, 1) + timedelta(seconds=int(pace_average)), "%H:%M:%S")

In [33]:
df

,name,distance,date,city,country,duration_total_timedelta,gpxfilename,date_str,duration_total_sec,duration_km_timedelta,...,pace_timedelta,pace_timedelta_str,pace_dist,pace_dist_km,pace_average_sec,pace_average_timedelta,pace_average_timedelta_str,pace_average_calc_sec,pace_average_calc_timedelta,pace_average_calc_timedelta_str
0,Cursa Moritz Sant Antoni,5,2025-01-26,Barcelona,Spain,0 days 00:29:19,Cursa_Moritz_Sant_Antoni_5km_2025.gpx,"Jan 26, 2025",1759,0 days 00:05:51.800000,...,"[0:08:31, 0:05:58, 0:05:53, 0:05:29, 0:05:22]","[00:08:31, 00:05:58, 00:05:53, 00:05:29, 00:05...","[1.001795896440534, 0.9996503427219456, 1.0015...","[1.001795896440534, 0.9996503427219456, 1.0015...",377,0:06:17,00:06:17,377,0:06:17,00:06:17
1,XIII Cursa Benefica Malalties Minoritaries,5,2025-02-23,Badalona,Spain,0 days 00:27:10,Cursa_Benèfica_Malalties_Minoritàries_Badalona...,"Feb 23, 2025",1630,0 days 00:05:26,...,"[0:05:42, 0:05:26, 0:05:32, 0:05:25, 0:05:16]","[00:05:42, 00:05:26, 00:05:32, 00:05:25, 00:05...","[1.002607244799694, 1.0017055420562166, 0.9990...","[1.002607244799694, 1.0017055420562166, 0.9990...",331,0:05:31,00:05:31,331,0:05:31,00:05:31
2,Cursa de la Dona Pienda,5,2025-03-23,Pineda de Mar,Spain,0 days 00:25:36,Cursa_de_la_Dona_Pineda_de_Mar_5_Km.gpx,"Mar 23, 2025",1536,0 days 00:05:07.200000,...,"[0:05:40, 0:05:07, 0:05:02, 0:04:53, 0:04:56]","[00:05:40, 00:05:07, 00:05:02, 00:04:53, 00:04...","[1.0023675457042622, 0.999403936365002, 0.9984...","[1.0023675457042622, 0.999403936365002, 0.9984...",311,0:05:11,00:05:11,311,0:05:11,00:05:11
3,15a Cursa Nocturna de L'Hospitalet,5,2025-04-05,L'Hospitalet de Llobregat,Spain,0 days 00:25:40,Cursa_Nocturna_de_L_Hospitalet_5_Km.gpx,"Apr 05, 2025",1540,0 days 00:05:08,...,"[0:05:35, 0:05:24, 0:05:13, 0:04:37, 0:04:44]","[00:05:35, 00:05:24, 00:05:13, 00:04:37, 00:04...","[1.002025532874051, 1.0007602062246772, 1.0005...","[1.002025532874051, 1.0007602062246772, 1.0005...",310,0:05:10,00:05:10,310,0:05:10,00:05:10
4,Cursa Montilivi Girona,5,2025-04-27,Girona,Spain,0 days 00:28:49,Cursa_Montilivi_Girona_5_Km.gpx,"Apr 27, 2025",1729,0 days 00:05:45.800000,...,"[0:04:45, 0:04:36, 0:05:05, 0:04:57, 0:04:55]","[00:04:45, 00:04:36, 00:05:05, 00:04:57, 00:04...","[1.0000387027949456, 1.0016146747205352, 1.001...","[1.0000387027949456, 1.0016146747205352, 1.001...",297,0:04:57,00:04:57,297,0:04:57,00:04:57
5,La Cursa de RAC1 Port de Barcelona,6,2025-05-18,Barcelona,Spain,0 days 00:30:04,Cursa_de_RAC1_Port_de_Barcelona_6Km.gpx,"May 18, 2025",1804,0 days 00:05:00.666666666,...,"[0:05:21, 0:04:42, 0:04:41, 0:04:54, 0:04:57, ...","[00:05:21, 00:04:42, 00:04:41, 00:04:54, 00:04...","[1.00014675059879, 1.0023605972350642, 0.99793...","[1.00014675059879, 1.0023605972350642, 0.99793...",296,0:04:56,00:04:56,296,0:04:56,00:04:56
6,14a Cursa Popular Illa Carlemany,5,2025-06-01,Andorra la Vella,Andorra,0 days 00:25:12,Cursa_Popular_Illa_Carlemany_Andorra_5_Km.gpx,"Jun 01, 2025",1512,0 days 00:05:02.400000,...,"[0:05:24, 0:04:30, 0:04:55, 0:04:39, 0:04:59]","[00:05:24, 00:04:30, 00:04:55, 00:04:39, 00:04...","[1.0059423014768352, 0.9940689054344989, 1.000...","[1.0059423014768352, 0.9940689054344989, 1.000...",294,0:04:54,00:04:54,294,0:04:54,00:04:54
7,45a Cursa Popular Ciutat de Badalona - La Curs...,5,2025-06-28,Badalona,Spain,0 days 00:23:40,La_Cursa_Del_Dimoni_Badalona_5Km.gpx,"Jun 28, 2025",1420,0 days 00:04:44,...,"[0:05:29, 0:04:33, 0:04:32, 0:04:43]","[00:05:29, 00:04:33, 00:04:32, 00:04:43]","[1.0029461093215961, 1.0013265293656526, 1.001...","[1.0029461093215961, 1.0013265293656526, 1.001...",288,0:04:48,00:04:48,288,0:04:48,00:04:48


In [40]:
def from_str_to_timedelta_pace(row):
    duration = datetime.strptime(row["pace_average_official_timedelta_str"], "%H:%M:%S")
    duration = timedelta(hours=duration.hour,
                         minutes=duration.minute,
                         seconds=duration.second)
    return duration

df = df.rename(columns={"pace": "pace_average_official_timedelta_str"})
df["pace_average_official_timedelta"] = df.apply(from_str_to_timedelta_pace, axis=1)
df["pace_average_official_sec"] = df.apply(lambda row: int(row["pace_average_official_timedelta"].total_seconds()), axis=1)

In [41]:
df

,name,distance,date,city,country,duration,pace_average_official_timedelta_str,gpxfilename,pace_average_official_timedelta,pace_average_official_sec
0,Cursa Moritz Sant Antoni,5,2025-01-26,Barcelona,Spain,0:29:19,0:05:52,Cursa_Moritz_Sant_Antoni_5km_2025.gpx,0 days 00:05:52,352
1,XIII Cursa Benefica Malalties Minoritaries,5,2025-02-23,Badalona,Spain,0:27:10,0:05:26,Cursa_Benèfica_Malalties_Minoritàries_Badalona...,0 days 00:05:26,326
2,Cursa de la Dona Pienda,5,2025-03-23,Pineda de Mar,Spain,0:25:36,0:05:08,Cursa_de_la_Dona_Pineda_de_Mar_5_Km.gpx,0 days 00:05:08,308
3,15a Cursa Nocturna de L'Hospitalet,5,2025-04-05,L'Hospitalet de Llobregat,Spain,0:25:40,0:05:08,Cursa_Nocturna_de_L_Hospitalet_5_Km.gpx,0 days 00:05:08,308
4,Cursa Montilivi Girona,5,2025-04-27,Girona,Spain,0:28:49,0:05:46,Cursa_Montilivi_Girona_5_Km.gpx,0 days 00:05:46,346
5,La Cursa de RAC1 Port de Barcelona,6,2025-05-18,Barcelona,Spain,0:30:04,0:05:01,Cursa_de_RAC1_Port_de_Barcelona_6Km.gpx,0 days 00:05:01,301
6,14a Cursa Popular Illa Carlemany,5,2025-06-01,Andorra la Vella,Andorra,0:25:12,0:05:03,Cursa_Popular_Illa_Carlemany_Andorra_5_Km.gpx,0 days 00:05:03,303
7,45a Cursa Popular Ciutat de Badalona - La Curs...,5,2025-06-28,Badalona,Spain,0:23:40,0:04:44,La_Cursa_Del_Dimoni_Badalona_5Km.gpx,0 days 00:04:44,284
